# **Finetuned ResNet50**

## **Imports**

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from sklearn.preprocessing import LabelEncoder

from utils import load_images
from utils import TrainingPlot

## **Parameters**

In [13]:
MODEL_VERSION = "_resNet50_1.1"

METADATA = pd.read_csv('../src/meta_data.csv')
IMAGE_FOLDER = "../src/final_data"
IMG_HEIGHT = 128
IMG_WIDTH = 128
NUM_CLASSES = 19
FREEZED_LAYERS = 100
TEST_SPLIT = 0.2
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.01

## **Load Images**

In [14]:
METADATA = pd.read_csv('../src/meta_data.csv')

# Load images and labels
image_folder = '../src/final_data'
images, labels = load_images(image_folder, METADATA, IMG_HEIGHT, IMG_WIDTH)

# Normalize images
images = images / 255.0

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Train / Test Split
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

del images
del labels
del labels_categorical

Processing: 100%|██████████| 57141/57141 [01:11<00:00, 801.35it/s]


## **Create Model**

In [15]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = base_model.output
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=HeNormal())(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# freeze first x layers
for layer in base_model.layers[:FREEZED_LAYERS]: 
    layer.trainable = False
for layer in base_model.layers[FREEZED_LAYERS:]:  
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])


## **Training**

In [16]:
# Train the model
#checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='loss', save_best_only=True, mode='min')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, mode='min')

# Train the model
history = model.fit(X_train, y_train,
    validation_split=TEST_SPLIT,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, TrainingPlot(MODEL_VERSION)]
)

Epoch 1/50
 311/2286 [===>..........................] - ETA: 17:04 - loss: 3.0226 - accuracy: 0.0945

In [ ]:
model.save(f"../src/models/model_v{MODEL_VERSION}.h5")
model.save(f"../src/models/model_v{MODEL_VERSION}.keras")

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
del X_train
del X_test
del y_train
del y_test
del images
del labels
del label_encoder
del labels_encoded
del model